In [19]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import regex as re
import pandas as pd
import numpy as np
import joblib
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 1. Splitting & Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.base import BaseEstimator, TransformerMixin

# 2. Pipeline Construction
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# 3. Preprocessing Steps
from sklearn.impute import SimpleImputer   
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder 
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import PowerTransformer
import ast

# 4. Model (Modeling)
from sklearn.linear_model import LinearRegression

# 5. Metrics (Metrics)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 6. Custom Transformer
import sys
import os
project_root = os.path.abspath("../")
sys.path.append(project_root)

from src.Custom_Transformer import *



In [21]:
def save_chart(fig, step, stt, title):
    """
    Saves a COPY of the chart with transparent background, 
    leaving the original chart (on screen) untouched.
    """
    # 1. Path Setup
    folder_name = f"Plot_{step}"
    output_dir = os.path.join("plots", folder_name) 
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 2. CLONE the figure
    # This ensures we don't mess up the chart currently displayed in the notebook
    fig_export = go.Figure(fig)

    # 3. Apply Transparent Background ONLY to the exported version
    fig_export.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
    )

    # 4. Filename Cleaning
    safe_title = re.sub(r'[\\/*?:"<>|]', '', title).strip()
    filename_base = f"[{step}]_[{stt}]_{safe_title}"
    
    path_svg = os.path.join(output_dir, f"{filename_base}.svg")
    path_png = os.path.join(output_dir, f"{filename_base}.png")

    # 5. Export
    try:
        fig_export.write_image(path_svg)
        fig_export.write_image(path_png, scale=3, width=1100, height=500)
        print(f"✅ Saved: {filename_base} (Transparent)")
    except Exception as e:
        print(f"❌ Error: {e}")

## **Introduction**

This notebook's main goal is to evaluate how much model performance improves when moving from *raw data* to *fully processed and feature-engineered data*, as well as to obtain more reliable insights into which features most strongly influence anime scores.

The workflow is organized into three parts:

* **Part I: Baseline Model with Raw Data:** We train a simple **Linear Regression** model on minimally processed raw data. This baseline establishes the lower bound of model performance and highlights the limitations of noisy or inconsistent features.

* **Part II: Final Model with Processed Data:** We train a refined model using the transformed dataset produced in `01_DataPreprocessing`. This includes feature engineering, cleaning, encoding, and handling missing values—allowing the model to learn from higher-quality inputs.

* **Part III: Model Comparison & Insights:** We compare baseline and final models using metrics such as **R²** and **MAE**, and analyze feature importance to understand which variables have the strongest effect on anime Score.

This notebook ultimately demonstrates how proper preprocessing and feature engineering significantly enhance predictive accuracy and interpretability in the Score regression task.


## **Part I : Baseline Model with Raw Data**
*Goal: Train a simple **Linear Regression** model on minimally processed raw data.*


### **1. Basic Data Preparing**

In [2]:
# 1. Load Raw Data
path_raw = r'..\data\raw\anime-dataset-2023.csv'
df_raw = pd.read_csv(path_raw)
print('Dataset initial shape:', df_raw.shape)

# 2. Drop Unnecessary Columns
df_raw = df_raw.drop(['Licensors','Premiered', 'English name', 'Other name', 'Image URL','Synopsis','Rank','anime_id', 'Name','Popularity','Favorites','Scored By','Members'], axis = 1)

# 3. Standardized NaN and Drop all rows with NaN
# List of values to be treated as NaN (case-insensitive)
nan_like_values = ['unknown', 'not available', 'n/a', 'na', 'tbd', 'tba', '---']

# Iterate through object columns and replace NaN-like values
for col in df_raw.select_dtypes(include=['object']).columns:
    mask = df_raw[col].str.lower().isin(nan_like_values)
    df_raw.loc[mask.fillna(False), col] = np.nan

df_base = df_raw.dropna()

# 3. Change data type
numerical_cols = ['Score', 'Episodes']
for col in numerical_cols:
    df_base.loc[:, col] = pd.to_numeric(df_base[col], errors='coerce')

print('Dataset shape after basic preparing:', df_base.shape)


Dataset initial shape: (24905, 24)
Dataset shape after basic preparing: (7231, 11)


### **2. Splitting Train and Test Set (80/20)**

In [3]:
# 1. Split X (features) and y (target)
X_base = df_base.drop('Score', axis=1)
y_base = df_base['Score']

# 2. Perform the data split
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y_base, 
                                                    test_size=0.2, 
                                                    random_state=42)

print("Baseline shapes:")
print(f"X_train shape: {X_train_base.shape}")
print(f"X_test shape: {X_test_base.shape}")


Baseline shapes:
X_train shape: (5784, 10)
X_test shape: (1447, 10)


### **3. Baseline Model**

To compute baseline metrics on the raw dataset, all categorical (`object`) columns must be converted into numeric form.  
We use **Ordinal Encoding** as a minimal, lightweight transformation that:

- Keeps the raw structure intact  
- Avoids complex preprocessing  
- Allows Linear Regression to run on the original feature set  

This creates a fair baseline for comparing against the fully processed dataset later.


In [4]:
# 1. Ordinal encode for object features

object_cols = X_train_base.select_dtypes(include=['object']).columns.tolist()
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Fit encoder on train, then transform
X_train_enc = X_train_base.copy()
X_test_enc  = X_test_base.copy()

X_train_enc[object_cols] = encoder.fit_transform(X_train_base[object_cols])
X_test_enc[object_cols]  = encoder.transform(X_test_base[object_cols])

# 2. Train Linear Regression baseline
model_base = LinearRegression()
model_base.fit(X_train_enc, y_train_base)

# 3. Predict
y_pred_base = model_base.predict(X_test_enc)

# 4. Evaluating Metrics
r2_base = r2_score(y_test_base, y_pred_base)
mae_base = mean_absolute_error(y_test_base, y_pred_base)

print("\n=== Baseline Linear Regression ===")
print(f"R²  : {r2_base:.4f}")
print(f"MAE : {mae_base:.4f}")




=== Baseline Linear Regression ===
R²  : 0.0887
MAE : 0.6386


## **Part II : Final Model with Prepared Data**
*Goal: train a refined model using the transformed dataset and pipeline produced in `01_DataPreprocessing`*


### **1. Load Prepared Data**

In [5]:
# 1. Load prepared_data
path = r'..\data\processed\prepared_data.csv'
df1 = pd.read_csv(path)

# Drop rows where target 'Score' is NaN, and Drop redundant column
print('Shape before drop Cols, Rows', df1.shape)
df1 = df1.dropna(subset=['Score'])
df1 = df1.drop(['Aired Date Start','Aired Date End','anime_id', 'Name','Rank','Popularity','Favorites','Scored By','Members'], axis= 1)
print('Shape after drop Cols, Rows', df1.shape)

# Example
df1.head(1)

Shape before drop Cols, Rows (23575, 21)
Shape after drop Cols, Rows (15692, 12)


,Score,Genres,Type,Episodes,Status,Producers,Studios,Source,Rating,Aired Year,Aired Month,Duration Minutes
0,8.75,"['Action', 'Award Winning', 'Sci-Fi']",TV,26.0,Finished Airing,['Bandai Visual'],['Sunrise'],Original,R - 17+ (Violence & Profanity),1998.0,4.0,24.0


### **2. Splitting Train and Test Set (80/20)**

In [6]:
# Split X (features) and y (target)
X = df1.drop('Score', axis=1)
y = df1['Score']

# Perform the data split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")


X_train shape: (12553, 11)
X_test shape: (3139, 11)


### **3. Final Model**

In [7]:
# 1. Load Pipeline
print("--- LOADING PIPELINE ---")

pipeline_filename = r"..\models\processing_pipeline.pkl"
full_pipeline = joblib.load(pipeline_filename)

print("Pipeline loaded successfully!")

--- LOADING PIPELINE ---
Pipeline loaded successfully!


In [8]:
# 2. Evaluating with Cross-Validation
full_pipe = Pipeline(
   [('processor', full_pipeline),
   ('model', LinearRegression())]
)

# 3. Evaluating with CV
kfold = KFold(n_splits = 3, shuffle=True, random_state=42)

# R2
cv_r2 = cross_val_score(full_pipe, X, y, cv=kfold, scoring="r2")

# MAE 
cv_mae = cross_val_score(full_pipe, X, y, cv=kfold, scoring="neg_mean_absolute_error")

print("\n=== Cross Validation (5-fold) ===")
print(f"R² scores: {cv_r2}")
print(f"Mean R²:   {cv_r2.mean():.4f}")
print(f"Std  R²:   {cv_r2.std():.4f}")

print("\nMAE scores (positive):", -cv_mae)
print(f"Mean MAE:  {-cv_mae.mean():.4f}")
print(f"Std  MAE:  {cv_mae.std():.4f}")


=== Cross Validation (5-fold) ===
R² scores: [0.52542491 0.53755528 0.5443771 ]
Mean R²:   0.5358
Std  R²:   0.0078

MAE scores (positive): [0.49073038 0.49001657 0.49144295]
Mean MAE:  0.4907
Std  MAE:  0.0006


In [9]:
# 3. Final Evaluation on Test Set

X_train_process = full_pipeline.transform(X_train)
X_test_process = full_pipeline.transform(X_test)

# Training Model
model = LinearRegression()
model.fit(X_train_process, y_train)
y_pred = model.predict(X_test_process)

# Evaluating Metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("\n=== Final Model ===")
print(f"R²  : {r2:.4f}")
print(f"MAE : {mae:.4f}")


=== Final Model ===
R²  : 0.5317
MAE : 0.4853


In [58]:
# 4. Save model in same folder as pipeline

model_path = r"..\models\model.pkl"
joblib.dump(model, model_path)

print("Model saved to:", model_path)


Model saved to: ..\models\model.pkl


## **Part III: Model Comparison & Insights**
*Goal: Demonstrates how proper preprocessing and feature engineering significantly enhance predictive accuracy and interpretation*


### **1. Evaluating Metrics**

The comparison chart clearly shows that the data preparation pipeline significantly improves model performance:

- **R² increases from 0.09 to 0.54**, indicating that the final model explains much more variance in the target variable after preprocessing, feature engineering, and encoding.
- **MAE drops from 0.64 to 0.48**, showing that prediction errors become smaller and more stable.
- The improvement suggests that raw data alone is insufficient for linear models; the engineered features, handling of categorical/multi-label variables, and robust scaling are essential for capturing meaningful relationships.


In [29]:
import pandas as pd
import plotly.express as px

def plot_metric(r2_base, r2, mae_base, mae):
    # --- 1. CREATE DATAFRAME ---
    data = {
        'Metric': ['R²', 'R²', 'MAE', 'MAE'],
        'Model': ['Baseline', 'Final', 'Baseline', 'Final'],
        'Value': [r2_base, r2, mae_base, mae]
    }
    df = pd.DataFrame(data)

    # --- 2. COLOR STRATEGY ---
    colors = {
        "Baseline": "#D3D3D3",  # Light Grey for neutral / baseline
        "Final": "#ea568c",     # Vibrant Pink for clean / final
    }

    # --- 3. CREATE PLOT ---
    fig = px.bar(
        df,
        x='Metric',
        y='Value',
        color='Model',
        color_discrete_map=colors,
        barmode='group',
        text=df['Value'].round(2),
        title="<b>Metrics Before and After Data Preparation</b><br>"
              "<span style='color:#555555; font-size:14px'>(Metrics Comparison in Baseline and Final Models)</span>"
    )

    # --- 4. LAYOUT SETUP (General Settings) ---
    fig.update_layout(
        font=dict(family="Tahoma", size=12, color="black"),
        template="plotly_white",
        margin=dict(l=40, r=40, t=100, b=60),
        title_x=0.02,
        hovermode="x unified",
        height=500,
        width=800,
        showlegend=True,
        legend=dict(
            x=1.02, y=0.98,
            xanchor='left', yanchor='top',
            bgcolor='rgba(0,0,0,0)'
        ),
        bargap=0.5,
        bargroupgap=0.25
    )

    # --- 5. AXIS SETTINGS ---
    # X-axis: bold labels
    fig.update_xaxes(
        title_text="",
        ticktext=['<b>R²</b>', '<b>MAE</b>'],
        tickvals=['R²', 'MAE'],
        tickfont=dict(family="Tahoma", size=12, color="black")
    )

    # Y-axis: hide ticks
    fig.update_yaxes(
        title_text="",
        showticklabels=False,
        showgrid=False
    )

    # --- 6. TEXT ON BARS ---
    fig.update_traces(
        texttemplate='%{text:.2f}',
        textposition='outside'
    )
    try:
        save_chart(fig, step="ML_Models", stt="III_1", title="Metrics_Before_and_After_Data_Preparation")
    except Exception as e:
        print(f"⚠️ Could not save chart due to Kaleido error: {e}")
        print("✅ Chart is displayed below regardless.")

    fig.show()

# --- Example Usage ---
plot_metric(r2_base, r2, mae_base, mae)


✅ Saved: [ML_Models]_[III_1]_Metrics_Before_and_After_Data_Preparation (Transparent)


### **2. Feature Importance (based on Linear Regression Coefficients)**

After preprocessing, the model shifts from using weak signals (e.g., *Status*, *Rating*) in the baseline to much stronger and more meaningful features:

- `Producers` and `Studios` become the dominant predictors, showing that production teams carry major influence on anime quality and final scores.
- `Genres`, `Source`, and `Type` also gain importance once properly encoded.
- **Data preparation** unlocks the true predictive power of categorical and multi-label features, giving the final model much clearer and stronger feature signals than the baseline.

In [23]:
# Create Subplot with 1 row, 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        '<b>Baseline Model</b>',
        '<b>Final Model</b>'
    ),
    horizontal_spacing=0.3,
    vertical_spacing=0.1
)

# === FIGURE 1: BASELINE MODEL ===
feature_name = X_train_base.columns.tolist()
coefs_base = model_base.coef_

coef_df_base = pd.DataFrame({
    "feature": feature_name,
    "coef": coefs_base,
})

coef_df_base["abs_coef"] = coef_df_base["coef"].abs()

original_features = ['Genres', 'Type', 'Episodes', 'Status', 'Producers', 
                     'Studios', 'Source', 'Rating', 'Aired Year', 'Aired Month', 'Duration Minutes']

feature_to_original_group_base = {}
for feature in feature_name:
    assigned = False
    for orig_feat in original_features:
        if (feature == orig_feat or 
            feature.startswith(orig_feat + '_') or 
            feature.startswith(orig_feat + '__') or
            orig_feat in feature):
            feature_to_original_group_base[feature] = orig_feat
            assigned = True
            break
    if not assigned:
        feature_to_original_group_base[feature] = "Other"

coef_df_base["original_group"] = coef_df_base["feature"].map(feature_to_original_group_base)
group_importance_base = coef_df_base.groupby("original_group")["abs_coef"].sum().sort_values(ascending=False)
group_importance_df_base = group_importance_base.head(5).reset_index()
group_importance_df_base.columns = ["original_group", "total_importance"]

# Add baseline bar chart
fig.add_trace(
    go.Bar(
        x=group_importance_df_base['total_importance'],
        y=group_importance_df_base['original_group'],
        orientation='h',
        marker=dict(
            color=group_importance_df_base['total_importance'],
            colorscale=[(0, "#f1e6fc"), (0.8, "#76159c"), (1, "#570889")],
            showscale=False,
            line=dict(width=0)
        ),
        name='Baseline',
        hovertemplate='<b>%{y}</b><br>Importance: %{x:.4f}<extra></extra>',
        text=group_importance_df_base['total_importance'].round(3),
        textposition='outside',
        textfont=dict(size=14),
        insidetextanchor='start',
        cliponaxis=False,  
    ),
    row=1, col=1
)

# === FIGURE 2: FINAL MODEL ===
try:
    feature_names = full_pipeline.named_steps["preprocessor"].get_feature_names_out()
except:
    feature_names = np.array([f"feat_{i}" for i in range(X_train_process.shape[1])])

coefs = model.coef_

coef_df_final = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs,
})

coef_df_final["abs_coef"] = coef_df_final["coef"].abs()

feature_to_original_group_final = {}

for feature in feature_names:

    # NEW RULE → Aired Year + Aired Month + SeasonCat → Aired Time Features
    if ("Aired Year" in feature) or ("Aired Month" in feature):
        feature_to_original_group_final[feature] = "Aired Time Features"
        continue

    assigned = False

    # Existing custom rules
    if 'Aired Year_deg' in feature:
        feature_to_original_group_final[feature] = 'Aired Time Features'
        continue

    elif any(x in feature for x in ['Genres_Count', 'Genres__']):
        feature_to_original_group_final[feature] = 'Genres'
        continue

    elif any(x in feature for x in ['Producers_Count', 'Producers__']):
        feature_to_original_group_final[feature] = 'Producers'
        continue

    elif any(x in feature for x in ['Studios_Count', 'Studios__']):
        feature_to_original_group_final[feature] = 'Studios'
        continue

    elif 'Episodes_x_Duration Minutes' in feature:
        feature_to_original_group_final[feature] = 'Episodes, Duration Minutes'
        continue

    elif 'EpisodesCat' in feature:
        feature_to_original_group_final[feature] = 'Episodes'
        continue

    elif 'DurationCat' in feature:
        feature_to_original_group_final[feature] = 'Duration Minutes'
        continue

    elif any(x in feature for x in ['Aired Month_sin', 'Aired Month_cos']):
        feature_to_original_group_final[feature] = 'Aired Time Features'
        continue

    # Fallback rules
    for orig_feat in original_features:
        if (feature == orig_feat or 
            feature.startswith(orig_feat + '_') or 
            feature.startswith(orig_feat + '__') or
            orig_feat in feature):
            feature_to_original_group_final[feature] = orig_feat
            assigned = True
            break

    if not assigned:
        feature_to_original_group_final[feature] = "Other"

coef_df_final["original_group"] = coef_df_final["feature"].map(feature_to_original_group_final)
group_importance_final = coef_df_final.groupby("original_group")["abs_coef"].sum().sort_values(ascending=False)
group_importance_df_final = group_importance_final.head(5).reset_index()
group_importance_df_final.columns = ["original_group", "total_importance"]




# --- 1. CREATE SUBPLOT ---
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "<b>Baseline Model</b>", 
        "<b>Final Model</b>"
    ),
    horizontal_spacing=0.18
)

# --- 2. COLOR STRATEGY ---
colors = {
    "Red": "#C3122F",        # Baseline
    "PinkStart": "#ffb6c1",  # Gradient start for Final
    "PinkEnd": "#ea568c",    # Gradient end for Final
    "LightGrey": "#D3D3D3"   # Neutral / Other
}

# --- 3. BASELINE BAR (All Red) ---
fig.add_trace(
    go.Bar(
        x=group_importance_df_base['total_importance'],
        y=group_importance_df_base['original_group'],
        orientation='h',
        marker_color=colors['Red'],
        text=group_importance_df_base['total_importance'].round(2),
        textposition='outside',
        textfont=dict(family="Tahoma", size=12),
        hovertemplate='%{y}<br>Importance: %{x:.2f}<extra></extra>',
        name='Baseline'
    ),
    row=1, col=1
)

# --- 4. FINAL BAR (Gradient Pink) ---
# Normalizing to 0-1 for gradient mapping
vals = group_importance_df_final['total_importance']
norm_vals = (vals - vals.min()) / (vals.max() - vals.min())

final_colors = [f"rgba(234,86,140,{0.3 + 0.7*v})" for v in norm_vals]  # alpha gradient

fig.add_trace(
    go.Bar(
        x=group_importance_df_final['total_importance'],
        y=group_importance_df_final['original_group'],
        orientation='h',
        marker_color=final_colors,
        text=group_importance_df_final['total_importance'].round(2),
        textposition='outside',
        textfont=dict(family="Tahoma", size=12),
        hovertemplate='%{y}<br>Importance: %{x:.2f}<extra></extra>',
        name='Final'
    ),
    row=1, col=2
)

# --- 5. GENERAL LAYOUT ---
fig.update_layout(
    font=dict(family="Tahoma", size=12, color="black"),
    template="plotly_white",
    margin=dict(l=40, r=40, t=100, b=60),
    title=dict(
        text="<b>Feature Importance Before and After Data Preparation</b><br>"
             "<span style='color:#555555; font-size:14px'>(Feature Importance Comparison in Baseline and Final Models)</span>",
        x=0.02, y=0.95,
        font=dict(family="Tahoma", size=18)
    ),
    hovermode="x unified",
    height=500,
    width=1100,
    showlegend=False
)

# --- 6. AXIS SETTINGS ---
max_x = max(group_importance_df_base['total_importance'].max(), 
            group_importance_df_final['total_importance'].max()) * 1.15

fig.update_xaxes(range=[0, max_x], showgrid=False, row=1, col=1, title_text="",showticklabels=False)
fig.update_xaxes(range=[0, max_x], showgrid=False, row=1, col=2, title_text="",showticklabels=False)

fig.update_yaxes(autorange='reversed', tickfont=dict(family="Tahoma", size=12), row=1, col=1)
fig.update_yaxes(autorange='reversed', tickfont=dict(family="Tahoma", size=12), row=1, col=2)

# --- 7. BARGAP ---
fig.update_layout(bargap=0.3)

# --- 8. SHOW FIGURE ---
try:
    save_chart(fig, step="ML_Models", stt="III_2", title="Feature_Importance_Before_After_Data_Preparation")
except Exception as e:
    print(f"⚠️ Could not save chart due to Kaleido error: {e}")
    print("✅ Chart is displayed below regardless.")

fig.show()


✅ Saved: [ML_Models]_[III_2]_Feature_Importance_Before_After_Data_Preparation (Transparent)
